In [ ]:
from pathlib import Path
from transformers import BartConfig, BartTokenizer, BartForConditionalGeneration

model_path = Path('../bart_large_cnn')

def load_tokenizer():
    tokenizer = BartTokenizer.from_pretrained(str(model_path))
    config = BartConfig.from_pretrained(str(model_path))
    config.output_hidden_states = True
    return tokenizer, config

def load_model(config):
    model = BartForConditionalGeneration.from_pretrained(model_path, config=config)
    model = model.cuda()
    return model

In [ ]:
tokenizer, config = load_tokenizer()
model = load_model(config)

In [ ]:
import torch

with open("test.txt", "r") as f:
    sample = f.read()

print(len(sample.split()), ' words')

sentences = sample.split(". ")
concatenated = []

for sentence in sentences:
    words = sentence.split()
    if not concatenated:
        concatenated.append(sentence)
    elif len(concatenated[-1].split()) + len(words) < 720:
        concatenated[-1] += ". " + sentence
    else:
        concatenated.append(sentence)

for i, sentence in enumerate(concatenated):
    print(i, len(sentence.split()))

device = "cuda:0" if torch.cuda.is_available() else "cpu"
inputs = tokenizer(concatenated, max_length=1024, return_tensors="pt", padding="max_length", truncation=True, ).to(device)

In [ ]:
from datetime import datetime

time_start = datetime.now()
summary_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=12, max_length=72, early_stopping=True)
outputs = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
time_end = datetime.now()
print('Time cost: ', (time_end - time_start).seconds, ' seconds')

for i, output in enumerate(outputs):
    print("{}: {} \n\n".format(i+1, output))